In [ ]:
%pip install --upgrade pip -q

In [ ]:
%pip install 'autogen-agentchat==0.4.0.dev8' -qU
%pip install 'autogen-ext[openai,azure]==0.4.0.dev8' -qU

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../../../.env")
os.environ.get("AZURE_OPENAI_ENDPOINT_GPT_4o")

api_key = os.environ["AZURE_OPENAI_API_KEY_GPT_4o"]
api_base = os.environ["AZURE_OPENAI_ENDPOINT_GPT_4o"]

## Selector Group Chat
`SelectorGroupChat` implements a team where participants take turns broadcasting messages to all other participants, with the next speaker selected by a generative model (e.g., an LLM) based on the shared context. This enables dynamic and context-aware multi-agent collaboration.

SelectorGroupChat provides several key features:

- Model-based speaker selection

- Configurable participant roles and descriptions

- Optional prevention of consecutive turns by the same speaker

- Customizable selection prompting

- Customizable selection function to override the default model-based selection

### How does it work?
SelectorGroupChat is a group chat similar to RoundRobinGroupChat, but with a model-based next speaker selection mechanism. When the team receives a task through run() or run_stream(), the following steps are executed:

- The team analyzes the current conversation context, including the conversation history and participants’ name and description attributes, to determine the next speaker using a model. You can override the model by providing a custom selection function.

- The team prompts the selected speaker agent to provide a response, which is then broadcasted to all other participants.

- The termination condition is checked to determine if the conversation should end, if not, the process repeats from step 1.

- When the conversation ends, the team returns the TaskResult containing the conversation history from this task.

Once the team finishes the task, the conversation context is kept within the team and all participants, so the next task can continue from the previous conversation context. You can reset the conversation context by calling `reset()`.

In this section, we will demonstrate how to use `SelectorGroupChat` with a simple example for a web search and data analysis task.

## Web Search and Analysis Example

### Agents
<Image src='docs/selector-group-chat.svg'>

In [2]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [3]:
from typing import Sequence

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import AgentMessage
from autogen_agentchat.task import Console, MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models import AzureOpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=api_base,
    model="gpt-4o",
    azure_deployment="gpt-4o-lei",
    api_key=api_key,
    api_version="2024-06-01"
)


planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        Web search agent: Searches for information
        Data analyst: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="A web search agent.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="A data analyst agent. Useful for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    """,
)

In [4]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
)

In [6]:
await team.reset()  # Reset the team for the next run.

In [ ]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

## Custom Selector Function
Often times we want better control over the selection process. To this end, we can set the `selector_func` argument with a custom selector function to override the default model-based selection. For instance, we want the Planning Agent to speak immediately after any specialized agent to check the progress.

In [ ]:
def selector_func(messages: Sequence[AgentMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))

The `on_messages_stream()` method returns an asynchronous generator that yields each inner message generated by the agent, and the last item is the final response message in the `chat_message` attribute.

In [ ]:
from autogen_agentchat.task import Console

# Use `asyncio.run(single_agent_team.reset())` when running in a script.
await single_agent_team.reset()  # Reset the team for the next run.
# Use `asyncio.run(single_agent_team.run_stream(task="What is the weather in Seattle?"))` when running in a script.
await Console(
    single_agent_team.run_stream(task="What is the weather in Seattle?")
)  # Stream the messages to the console.

## Reflection Pattern
Now we will create a team with two agents that implements the Reflection pattern, which is a multi-agent design pattern that uses a critic agent to evaluate the responses of a primary agent.

See how the reflection pattern works using the Core API.

In this example, we will use the `AssistantAgent` agent class for both the primary and critic agents. We will use both the `TextMentionTermination` and `MaxMessageTermination` termination conditions together to stop the team.

In [10]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models import AzureOpenAIChatCompletionClient
from autogen_agentchat.task import MaxMessageTermination, TextMentionTermination

# Create an OpenAI model client.
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=api_base,
    model="gpt-4o",
    azure_deployment="gpt-4o-lei",
    api_key=api_key,
    api_version="2024-06-01"
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(5)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

# Create a team with the primary and critic agents.
reflection_team = RoundRobinGroupChat(
    [primary_agent, critic_agent], termination_condition=termination)

In [ ]:
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write a short poem about fall season.")))` when running in a script.
await Console(
    reflection_team.run_stream(task="Write a short poem about fall season.")
)  # Stream the messages to the console.

### Resuming Team
Let’s run the team again with a new task while keeping the context about the previous task.

In [ ]:
# Write the poem in Chinese Tang poetry style.
# Use `asyncio.run(Console(reflection_team.run_stream(task="将这首诗用中文唐诗风格写一遍。")))` when running in a script.
await Console(reflection_team.run_stream(task="将这首诗用中文唐诗风格写一遍。"))

In [ ]:
# Write the poem in Spanish.
# Use `asyncio.run(Console(reflection_team.run_stream(task="Write the poem in Spanish.")))` when running in a script.
await Console(reflection_team.run_stream(task="Write the poem in Spanish."))

### Resuming A Previous Task
We can call `run()` or `run_stream()` methods without setting the task again to resume the previous task. The team will continue from where it left off.

In [ ]:
# Use the `asyncio.run(Console(reflection_team.run_stream()))` when running in a script.
await Console(reflection_team.run_stream())

## Pause for User Input
Often times, team needs additional input from the application (i.e., user) to continue processing the task. We will show two possible ways to do it:

- Set the maximum number of turns such that the team stops after the specified number of turns.

- Use the `HandoffTermination` termination condition.

You can also use custom termination conditions, see Termination Conditions.

### Maximum Number of Turns
This is the simplest way to pause the team for user input. For example, you can set the maximum number of turns to 1 such that the team stops right after the first agent responds. This is useful when you want the user to constantly engage with the team, such as in a chatbot scenario.

Simply set the `max_turns` parameter in the `RoundRobinGroupChat()` constructor.

```python
team = RoundRobinGroupChat([...], max_turns=1)
```

Once the team stops, the turn count will be reset. When you resume the team, it will start from 0 again.

Note that `max_turn` is specific to the team class and is currently only supported by `RoundRobinGroupChat`, `SelectorGroupChat`, and `Swarm`. When used with termination conditions, the team will stop when either condition is met.

### Using Handoff to Pause Team
You can use the `HandoffTermination` termination condition to stop the team when an agent sends a `HandoffMessage` message.

Let’s create a team with a single `AssistantAgent` agent with a handoff setting.

In [15]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent, Handoff
from autogen_ext.models import AzureOpenAIChatCompletionClient
from autogen_agentchat.task import HandoffTermination, TextMentionTermination


# Create an OpenAI model client.
model_client = AzureOpenAIChatCompletionClient(
    azure_endpoint=api_base,
    model="gpt-4o",
    azure_deployment="gpt-4o-lei",
    api_key=api_key,
    api_version="2024-06-01"
)

# Create a lazy assistant agent that always hands off to the user.
lazy_agent = AssistantAgent(
    "lazy_assistant",
    model_client=model_client,
    handoffs=[Handoff(target="user", message="Transfer to user.")],
    system_message="Always transfer to user when you don't know the answer. Respond 'TERMINATE' when task is complete.",
)

# Define a termination condition that checks for handoff message targetting helper and text "TERMINATE".
handoff_termination = HandoffTermination(target="user")
text_termination = TextMentionTermination("TERMINATE")
termination = handoff_termination | text_termination

# Create a single-agent team.
lazy_agent_team = RoundRobinGroupChat(
    [lazy_agent], termination_condition=termination)

In [ ]:
from autogen_agentchat.task import Console

# Use `asyncio.run(Console(lazy_agent_team.run_stream(task="What is the weather in New York?")))` when running in a script.
await Console(lazy_agent_team.run_stream(task="What is the weather in New York?"))

In [ ]:
# Use `asyncio.run(Console(lazy_agent_team.run_stream(task="It is raining in New York.")))` when running in a script.
await Console(lazy_agent_team.run_stream(task="It is raining in New York."))